In [1]:
import pandas as pd
import numpy as np

## Data Source
- "data/DB_Indicators.xlsx" => World Bank's economics indicators  
The last 5 rows are footer, so they will be dropped.

In [2]:
df = pd.read_excel("data/DB_Indicators.xlsx")[:-5]

## Removing outliers by existing values
1. If 2017 is NaN, fill it with 2018's values
2. If 2018 is NaN, fill it with 2017's values
3. If 2019 is NaN, fill it with the mean of 2017 & 2018

In [3]:
df.loc[df["2017"].isnull(), "2017"] = df["2018"]
df.loc[df["2018"].isnull(), "2018"] = df["2017"]
df.loc[df["2019"].isnull(), "2019"] = (df["2017"] + df["2018"]) /2

## Calculating the mean with values from 2017 to 2019

In [4]:
df["Mean"] = df.mean(numeric_only=True, axis=1)

In [5]:
directory_path = "./"
file_name = "cleaned_DB_Indicators.xlsx"
full_directory_path = directory_path + file_name
writer = pd.ExcelWriter(full_directory_path, engine="xlsxwriter")
df.to_excel(writer, sheet_name="World Indicator", index=False)
writer.save()

## Extract column names and countries name

In [6]:
series_name = df["Series Name"][0:14].to_list()
countries_name = df["Country Name"][::14].to_list()

## Extact each 14 data of a country into a list

In [7]:
rows = []
data = []
for index, row in df.iterrows():
    data.append(row["Mean"])
    
    if (index+1) % 14 == 0 and index != 0:
        rows.append(data)
        data = []

In [8]:
df_aggregated = pd.DataFrame(data=rows, index=countries_name, columns=series_name)

In [9]:
directory_path = "./"
file_name = "aggregated_DB_Indicators.xlsx"
full_directory_path = directory_path + file_name
writer = pd.ExcelWriter(full_directory_path, engine="xlsxwriter")
df_aggregated.to_excel(writer, sheet_name="World Indicator")
writer.save()

## Data Source
- "data/DB_Nutrition.xlsx" => World Bank's healtlh indicators  
The last 5 rows are footer, so they will be dropped.

In [10]:
df_n = pd.read_excel("data/DB_Nutrition.xlsx")[:-5]

In [11]:
df_n.loc[df["2017"].isnull(), "2017"] = df["2018"]
df_n.loc[df["2018"].isnull(), "2018"] = df["2017"]
df_n.loc[df["2019"].isnull(), "2019"] = (df["2017"] + df["2018"]) /2
df_n["Mean"] = df.mean(numeric_only=True, axis=1)

In [12]:
nutrition_series_name = df_n["Series Name"][:9].to_list()
nutrition_countries_name = df_n["Country Name"][::9].to_list()
nutrition_countries_name == countries_name

True

In [13]:
rows = []
data = []
for index, row in df_n.iterrows():
    data.append(row["Mean"])
    
    if (index+1) % 9 == 0 and index != 0:
        rows.append(data)
        data = []

In [14]:
df_aggregated_n = pd.DataFrame(data=rows, index=countries_name, columns=nutrition_series_name)

In [15]:
aggregated_result = pd.concat([df_aggregated, df_aggregated_n], axis=1, join="inner")

# These columns are all null
columns_to_be_dropped = [
    "Current health expenditure (% of GDP)",
    "Domestic general government health expenditure per capita, PPP (current international $)",
    "Nitrous oxide emissions (thousand metric tons of CO2 equivalent)",
    "Rail lines (total route-km)", # only 67 are un-null
]
aggregated_result.drop(columns_to_be_dropped, axis=1, inplace=True) 

In [16]:
directory_path = "./"
file_name = "aggregated_all.xlsx"
full_directory_path = directory_path + file_name
writer = pd.ExcelWriter(full_directory_path, engine="xlsxwriter")
aggregated_result.to_excel(writer, sheet_name="All indicators")
writer.save()

## Confirmed cases

In [17]:
df_confirmed = pd.read_csv("data/confirmed_cases.csv")

In [18]:
hk_index = df_confirmed.loc[df_confirmed["Province"] == "Hong Kong"]
macau_index = df_confirmed.loc[df_confirmed["Province"] == "Macau"]


Hong Kong and Macau shall be seperated from China in the calculation.

In [19]:
df_confirmed.loc[hk_index.index, "Country"] = "Hong Kong SAR, China"
df_confirmed.loc[macau_index.index, "Country"] = "Macao SAR, China"

In [20]:
df_confirmed["Total Confirmed"] = df_confirmed.groupby(["Country"])["Confirmed"].transform("sum")

In [21]:
df_c = df_confirmed.drop_duplicates("Country")

In [22]:
del df_c["Province"]

In [23]:
del df_c["Confirmed"]

## Death cases

In [24]:
df_death = pd.read_csv("data/death_cases.csv")
hk_index = df_death.loc[df_death["Province"]=="Hong Kong"]
macau_index = df_death.loc[df_death["Province"]=="Macau"]

df_death.loc[hk_index.index, "Country"] = "Hong Kong SAR, China"
df_death.loc[macau_index.index, "Country"]="Macao SAR, China"

In [25]:
df_death["Total Death"]=df_death.groupby(["Country"])["Death"].transform("sum")

In [26]:
df_d = df_death.drop_duplicates("Country")
del df_d["Province"]
del df_d["Death"]

In [27]:
aggregated_cases = pd.merge(df_c, df_d, on="Country")

In [28]:
aggregated_cases = aggregated_cases.set_index(keys=aggregated_cases["Country"], drop=True)
del aggregated_cases["Country"]

In [29]:
# country_diff = set(countries_name) - set(aggregated_cases["Country"])
# draft_df = pd.DataFrame([countries_name, aggregated_cases["Country"], sorted(list(country_diff))])

## Standardize country names in two DataFrame
aggregated_result / aggregrated_cases
13. Bahamas, The / **Bahamas**
44. Congo, Dem. Rep. / **Congo (Brazzaville)**
45. Congo, Rep. / **Congo (Kinshasa)**
52. **Czech Republic** / Czechia
58. Eygpt, Arab Rep. / **Eygpt**
71. Gambia, The / **Gambia**
91. Iran, Islamic Rep. / **Iran**
104. Korea, Rep. / **Korea, South**
107. **Kyrgyz Republic** / Kyrgyzstan
108. Lao PDR / **Laos**
161. Russian Federation / **Russia**
173. **Slovak Republic** / Slovakia
181. **St. Kitts and Nevis** / Saint Kitts and Nevis
182. **St. Lucia** / Saint Lucia
184. **St. Vincent and the Grenadines** / Saint Vincent and the Grenadines
189. Syrian Arab Republic / **Syria**
206. **United States** / US
210. Venezuela, RB / Venezula

In [30]:
result_dict = {
    "Bahamas, The": "Bahamas",
    "Congo, Demp. Rep": "Congo (Brazzaville)",
    "Congo, Rep.": "Congo (Kinshasa)",
    "Eygpt, Arab Rep.": "Eygpt",
    "Gambia, The": "Gambia",
    "Iran, Islamic Rep.": "Iran",
    "Korea, Rep.": "Korea, South",
    "Lao PDR": "Laos",
    "Russian Federation": "Russia",
    "Syrian Arab Republic": "Syria",
    "Venezuela, RB": "Venezula",
}

aggregated_result.rename(index=result_dict, inplace=True)

case_dict = {
    "Czechia": "Czech Republic",
    "Kyrgyzstan": "Kyrgyz Republic",
    "Slovakia": "Slovak Republic",
    "Saint Kitts and Nevis": "St. Kitts and Nevis",
    "Saint Lucia": "St. Lucia",
    "Saint Vincent and the Grenadines": "St. Vincent and the Grenadines",
    "US": "United States"
}

aggregated_cases.rename(index=case_dict, inplace=True)

In [31]:
combined_df = pd.merge(aggregated_cases, aggregated_result, left_index=True, right_index=True).sort_index()

In [32]:
directory_path = "./"
file_name = "combined_all.xlsx"
full_directory_path = directory_path + file_name
writer = pd.ExcelWriter(full_directory_path, engine="xlsxwriter")
combined_df.to_excel(writer, sheet_name="All indicators")
writer.save()

In [33]:
filled_combined_df = combined_df.fillna(combined_df.mean())

## All columns' name
~~~
0. 'Total Confirmed',
1. 'Total Death',
2. 'Adjusted net national income per capita (current US$)',
3. 'UHC service coverage index',
4. 'International tourism, number of arrivals',
5. 'Fixed broadband subscriptions (per 100 people)',
6. 'Fixed telephone subscriptions (per 100 people)',
7. 'Mobile cellular subscriptions (per 100 people)',
8. 'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)',
9. 'Population density (people per sq. km of land area)',
10. 'Urban population (% of total population)',
11. 'GDP per capita (current US$)',
12. 'GNI per capita, Atlas method (current US$)',
13. 'Literacy rate, adult total (% of people ages 15 and above)',
14. 'Public spending on education, total (% of GDP)',
15. 'Community health workers (per 1,000 people)',
16. 'Number of people who are undernourished',
17. 'Population ages 65 and above, total',
18. 'Population ages 0-14 (% of total population)',
19. 'People using at least basic drinking water services (% of population)',
20. 'People practicing open defecation (% of population)']`
~~~

In [34]:
sanitized_col_names = [
    "Total_Confirmed",
    "Total_Death",
    "National_Income_Per_Capital",
    "UHC_Service_Coverage_Index",
    "International_Tourism Arrivals",
    "Fixed_Broadband_Subscriptions",
    "Fixed_Telephone_Subscriptions",
    "Mobile_Phone_Subscriptions",
    "PM2.5_Mean_Annual_Exposure",
    "Population_Density",
    "Urban_Population",   #10
    "GDP_Per_Capital",
    "GNI_Per_Capital",
    "Adult_Literacy_Rate",
    "Public_Education_Expenses",
    "Community_Health_Workers",
    "Number_of_Undernourished",
    "Population_over_65",
    "Population_below_14",
    "People_with_Basic_Drinking_Water",
    "People_Practicing_Open_Defecation",
]

In [35]:
filled_combined_df.columns = sanitized_col_names

In [36]:
directory_path = "./"
file_name = "filled_combined_all.xlsx"
full_directory_path = directory_path + file_name
writer = pd.ExcelWriter(full_directory_path, engine="xlsxwriter")
filled_combined_df.to_excel(writer, sheet_name="All indicators")
writer.save()

In [37]:
cols = filled_combined_df.columns.to_list()

In [66]:
import statsmodels.formula.api as smf

def calculate_linear_regression(dep_var, indep_var):
    formula_str = f"{dep_var} ~ "
    if len(indep_var) == 1:
        formula_str += indep_var[0][0]
    else:
        s = " + ".join(indep_var)
        formula_str += s
        
    print(formula_str)
    
    result = smf.ols(
        formula=formula_str,
        data=filled_combined_df
    ).fit()
    
    print(f"R Square: {result.rsquared}\nP Values: \n{result.pvalues}")
    

# calculate_linear_regression(cols[1], [cols[2], cols[3]])

    
    
# result = smf.ols(
#     formula=f"{cols[1]} ~ {cols[2]}",
#     data=filled_combined_df
# ).fit()


In [67]:
formula_str = []
r_sqr = []
p_val = []

from itertools import combinations


for arg in range(1, 3):
    comb_list = list(combinations(cols, arg))
    print(comb_list)
    
    for comb in comb_list:
        calculate_linear_regression(cols[1], comb_list)

[('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',), ('q',), ('r',), ('s',), ('t',), ('u',)]


TypeError: sequence item 0: expected str instance, tuple found